In [1]:
# Importing libaries
import cv2
import matplotlib.pyplot as plt

In [2]:
# Threshold to detect object
thres = 0.56

In [3]:
#Labels
classNames= []
classFile = r'coco.names'
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

In [4]:
#loading pre-trained model from disk
configPath = r'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = r'frozen_inference_graph.pb'

In [5]:
#defining model
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

< cv2.dnn.Model 00000248387C6EF0>

### Testing model on Video

In [10]:
#loading video
writer = None
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
cap.set(10,70)
while True:
    success,img = cap.read()
    classIds, confs, bbox = net.detect(img,confThreshold=thres)
    #print(classIds,bbox)
    
    if len(classIds) != 0:
        for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
            cv2.rectangle(img,box,(0,0,0),1)
            cv2.putText(img,classNames[classId-1].upper(),(box[0]+10,box[1]+30),
                        cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)
    

    cv2.imshow("Output Video",img)
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file
        writer = cv2.VideoWriter('result.mp4', fourcc, 25,
                                 (img.shape[1], img.shape[0]), True)

    # Write processed current frame to the file
    writer.write(img)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        cv2.destroyAllWindows()
        break
#close all opencv windows open
cap.release()
cv2.destroyAllWindows()
writer.release()